In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
data_train=pd.read_csv('trainfile.csv')
data_test=pd.read_csv('testfile.csv')

In [3]:
PATH=os.getcwd()

train_dir = os.path.join(PATH,'Train')

test_dir = os.path.join(PATH,'Test')

BATCH_SIZE = 32
IMG_SIZE = (200, 200)


In [4]:
os.getcwd()

'C:\\Users\\91966\\Desktop\\ML\\hackathons\\snake challenge'

In [5]:
train_dir

'C:\\Users\\91966\\Desktop\\ML\\hackathons\\snake challenge\\Train'

In [6]:
data_train

,image_id,breed
0,a8b3ad1dde,nerodia-erythrogaster
1,8b492b973d,pantherophis-vulpinus
2,929b99ea92,thamnophis-sirtalis
3,bbac7385e2,pantherophis-obsoletus
4,ef776b1488,agkistrodon-contortrix
...,...,...
5503,5f0c970adf,lampropeltis-triangulum
5504,f6a4287fac,pantherophis-alleghaniensis
5505,d3c10e63e1,thamnophis-sirtalis
5506,61ef50879c,pituophis-catenifer


In [7]:
for i in range(len(data_train['image_id'])):
    data_train['image_id'][i]=data_train['image_id'][i]+'.jpg'
    
               

In [8]:
from keras.preprocessing import image

In [9]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Model
from keras.utils import to_categorical

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,)
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True,
        # width_shift_range=0.1,
        # height_shift_range=0.1

train_generator=train_datagen.flow_from_dataframe(dataframe=data_train,
                                            directory='Train',
                                            x_col="image_id",
                                            y_col="breed",
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(299,299))

Found 5508 validated image filenames belonging to 35 classes.


In [11]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.preprocessing import image

In [12]:
resnet=InceptionResNetV2(include_top=True,weights='imagenet')

# Customizing the final layer of Inception model

In [13]:
x=resnet.layers[-2].output
fc1=Dense(35,activation='softmax')(x)

In [14]:
my_model=Model(inputs=resnet.input,outputs=fc1)

In [15]:
from keras.optimizers import Adam

In [16]:
adam=Adam(learning_rate=0.004)

In [17]:
for l in my_model.layers[:-35]:
    #print(l)
    l.trainable = False
my_model.compile(optimizer='adam',loss ="categorical_crossentropy",metrics=["accuracy"])

In [26]:
my_model.fit_generator(train_generator,epochs=3,steps_per_epoch=5508//128)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
43/43 [==============================] - 121s 3s/step - loss: 2.6493 - accuracy: 0.2734
Epoch 2/3
43/43 [==============================] - 99s 2s/step - loss: 1.7810 - accuracy: 0.4710
Epoch 3/3
43/43 [==============================] - 100s 2s/step - loss: 1.2106 - accuracy: 0.6561


In [28]:
my_model.save('model.h5')

# Test Data

In [29]:
test_df=pd.read_csv('testfile.csv')

In [30]:
for i in range(len(test_df['image_id'])):
    test_df['image_id'][i]=test_df['image_id'][i]+'.jpg'

In [31]:
test_df.shape

(2361, 1)

In [32]:
test_datagen = ImageDataGenerator(rescale=1./255,)
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True,
        # width_shift_range=0.1,
        # height_shift_range=0.1

test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                            directory="Test",
                                            x_col="image_id",
                                            y_col=None,
                                            batch_size=128,
                                            seed=42,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(331,331))

Found 2361 validated image filenames.


In [33]:
y_pred=my_model.predict_generator(test_generator,
verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
19/19 [==============================] - 60s 3s/step


In [34]:
y_pred_2=np.argmax(y_pred,axis=1)

In [35]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred_2]

In [36]:
test_df=pd.read_csv('testfile.csv')

In [37]:
test_name=test_df['image_id'].values

In [38]:
data=pd.DataFrame((zip(test_name,predictions)),columns=['image_id','breed'])

In [39]:
data.to_csv('output.csv',index=False)

In [40]:
x_test=[]

In [41]:
img=image.load_img('Test/00a9bba384.jpg',target_size=(331,331))

In [42]:
img=image.img_to_array(img)/255
x_test.append(img)
x_test=np.array(x_test)
pre=my_model.predict(x_test)
pre=np.argmax(pre)

In [43]:
pre

32